In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [6]:
class ReviewLinkScrapper:
    def __init__(self):
        self.website = "https://www.amazon.in/"
        self.products_page = "https://www.amazon.in/gp/bestsellers/electronics/1805560031"
        self.product_links = []        
    
    def get_page_source(self,driver,url):
        driver.get(url)
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to bottom
            time.sleep(2) # Wait to load page
            new_height = driver.execute_script("return document.body.scrollHeight") # Calculate new scroll height
            if new_height == last_height:
                break
            last_height = new_height
        return driver.page_source
    
    def get_pagination(self):
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        page_source = self.get_page_source(driver,self.products_page)
        driver.close()
        soup = BeautifulSoup(page_source, 'lxml')
        starting_tag = soup.find_all('li',class_="a-selected")
        base_link = self.website + starting_tag[0].a['href'][:-1]
        ending_tag = soup.find_all('li',class_="a-last")
        pageNums = int(ending_tag[0].a['href'][-1])
        return (base_link,pageNums)
    
    def get_product_links(self,base,pageNums):
        product_links = []
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
        for i in range(pageNums):
            page_link = base+str(i+1) 
            page_source = self.get_page_source(driver,page_link)
            product_soup = BeautifulSoup(page_source, 'lxml')
            link_tags = product_soup.find_all('div',id="gridItemRoot")
            for i in range(len(link_tags)):
                product_links.append(self.website + link_tags[i].a['href'])
        driver.close()
        return product_links
    
    def get_review_links(self,product_links):
        all_review_links = []
        for link in product_links:
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.get(link)
            page_source = self.get_page_source(driver,link)
            soup = BeautifulSoup(page_source, 'lxml')
            link_of_all_reviews = soup.find_all('a',class_="a-link-emphasis a-text-bold")
            all_review_links.append(self.website+link_of_all_reviews[0]['href'])
            driver.close()
        return all_review_links
    
    def scrape_review_links(self):
        (base_link,page_number) = self.get_pagination()
        products = self.get_product_links(base_link,page_number)
        review_links = self.get_review_links(products)
        return review_links

In [7]:
review_links = ReviewLinkScrapper().scrape_review_links()

In [8]:
len(review_links)

100

In [9]:
review_links[0]

'https://www.amazon.in//OnePlus-Nord-Pastel-128GB-Storage/product-reviews/B0BY8JZ22K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [11]:
file = open('reviewlinks.txt','w')
for item in review_links:
    file.write(item+"\n")
file.close()

In [ ]:
class AmazonReviewScraper:
    def __init__(self,review_link):
        self.review_page = review_link
        